In [178]:
import numpy as np
from skimage.io import imread
from skimage.color import rgb2gray, rgb2hsv
from skimage.morphology import disk
from skimage.filters import threshold_otsu
from scipy.ndimage import binary_fill_holes
from skimage.morphology import opening
from skimage.measure import label, regionprops
from sklearn.model_selection import train_test_split


In [179]:
# Function to calculate color (hue in HSV color space)
def spalva_color(img):
    hsv_img = rgb2hsv(img)
    h_channel = hsv_img[:, :, 0]
    gray_img = rgb2gray(img)
    thresh = threshold_otsu(gray_img)
    bw = gray_img > thresh
    bw = binary_fill_holes(~bw)
    bw = opening(bw, disk(12))
    masked_hue = h_channel[bw]
    hsv_value = np.mean(masked_hue)
    return hsv_value


In [180]:
# Function to calculate roundness
def apvalumas_roundness(img):
    gray_img = rgb2gray(img)
    thresh = threshold_otsu(gray_img)
    bw = gray_img > thresh
    bw = binary_fill_holes(~bw)
    bw = opening(bw, disk(12))
    props = regionprops(label(bw))
    area = props[0].area
    perimeter = props[0].perimeter
    roundness = 4 * np.pi * area / (perimeter ** 2)
    return roundness

In [181]:
apple_images = ['apple_04.jpg','apple_05.jpg', 'apple_06.jpg', 'apple_07.jpg', 'apple_11.jpg', 'apple_12.jpg', 'apple_13.jpg', 'apple_17.jpg', 'apple_19.jpg']
pear_images = ['pear_02.jpg', 'pear_09.jpg', 'pear_01.jpg','pear_03.jpg' ]


In [182]:
# Compute features 
hsv_values_apples = [spalva_color(imread(img)) for img in apple_images]
roundness_apples = [apvalumas_roundness(imread(img)) for img in apple_images]
hsv_values_pears = [spalva_color(imread(img)) for img in pear_images]
roundness_pears = [apvalumas_roundness(imread(img)) for img in pear_images]

In [183]:
# Combine features and labels for classification
x1 = np.array(hsv_values_apples + hsv_values_pears)
x2 = np.array(roundness_apples + roundness_pears)    
features = np.column_stack((x1, x2)) 
labels = np.array([1] * len(apple_images) + [-1] * len(pear_images))  # 1 for apples, -1 for pears



In [184]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [185]:
# Initialize random weights and bias
w1 = np.random.randn()
w2 = np.random.randn()
b = np.random.randn()
eta = 0.1

In [186]:
# Classification before training 
for i in range(5): 
    v = x1[i] * w1 + x2[i] * w2 + b # percep rule
    if v > 0:
        y = 1  # Classified as apple.
    else:
        y = -1  # Classified as pear.
    print(f"Sample {i+1}: Classified as {'Apple' if y == 1 else 'Pear'}")

Sample 1: Classified as Apple
Sample 2: Classified as Apple
Sample 3: Classified as Apple
Sample 4: Classified as Apple
Sample 5: Classified as Apple


In [187]:
#training

max_epochs = 100  
for epoch in range(max_epochs):
    total_error = 0
    for i in range(len(X_train)):  
        v = X_train[i, 0] * w1 + X_train[i, 1] * w2 + b
        if v > 0:
            y = 1  # Classified as apple.
        else:
            y = -1  # Classified as pear.
        e = y_train[i] - y  
        total_error += abs(e)
        w1 += eta * e * X_train[i, 0]
        w2 += eta * e * X_train[i, 1]
        b += eta * e
    print(f"Epoch {epoch + 1}: Total Error = {total_error}")
    if total_error == 0:
        break


Epoch 1: Total Error = 8
Epoch 2: Total Error = 8
Epoch 3: Total Error = 8
Epoch 4: Total Error = 6
Epoch 5: Total Error = 8
Epoch 6: Total Error = 8
Epoch 7: Total Error = 6
Epoch 8: Total Error = 8
Epoch 9: Total Error = 6
Epoch 10: Total Error = 8
Epoch 11: Total Error = 2
Epoch 12: Total Error = 4
Epoch 13: Total Error = 0


In [188]:
#testing
correct = 0
for i in range(len(X_test)):
    v = X_test[i, 0] * w1 + X_test[i, 1] * w2 + b
    if v > 0:
        y = 1  # Classified as apple.
    else:
        y = -1  # Classified as pear.
    if y == y_test[i]:
        correct += 1

accuracy = correct / len(X_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")


Test accuracy: 100.00%
